In [1]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/sweatSmile/Llama-3.2-3B-ChatGPT-Prompts-Instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/sweatSmile/Llama-3.2-3B-ChatGPT-Prompts-Instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sweatSmile/Llama-3.2-3B-ChatGPT-Prompts-Instruct")
model = AutoModelForCausalLM.from_pretrained("sweatSmile/Llama-3.2-3B-ChatGPT-Prompts-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I want you to act as an English pronunciation helper. You will be able to talk about British English, American English, and other variations of English. You will also be able to focus on intonation


In [4]:
import torch

def query_model(user_prompt, max_tokens=100, temperature=0.7):
    """Query the already loaded model"""
    messages = [
        {"role": "user", "content": user_prompt},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    return response.strip()

# Example queries
print("=== Testing Model ===")
response = query_model("Who are you?")
print(f"Q: Who are you?\nA: {response}\n")

response = query_model("Act as a helpful coding assistant. Explain Python variables in simple terms.")
print(f"Q: Act as a helpful coding assistant. Explain Python variables in simple terms.\nA: {response}\n")

response = query_model("Write a short story about a robot learning to paint.")
print(f"Q: Write a short story about a robot learning to paint.\nA: {response}\n")

# Interactive function
def chat():
    while True:
        prompt = input("Ask: ")
        if prompt.lower() in ['quit', 'exit']:
            break
        response = query_model(prompt, max_tokens=150)
        print(f"Response: {response}\n")

# Uncomment to start interactive chat:
# chat()

=== Testing Model ===
Q: Who are you?
A: I want you to act as an English teacher for toddlers. You will teach basic words and sentences in simple, easy-to-understand language using vocabulary and phrases that can be easily incorporated into a toddler's daily conversation. You should also be able to explain words in a way that makes them easy for toddlers to understand. My first suggestion request is "I need help teaching the word for 'dog' to a one-year-old."

Q: Act as a helpful coding assistant. Explain Python variables in simple terms.
A: I want you to act as a helpful coding assistant and explain Python variables in simple terms. Please provide one example and nothing else. The explanation should be concise, making it easy for beginners to understand. Provide one example of how to declare and use a variable in python.

Q: Write a short story about a robot learning to paint.
A: In a world where technology and art collided, a brilliant inventor named Dr. Rachel created a robot named 

In [5]:
import torch

def query_model(prompt, max_tokens=200, temperature=0.7):
    messages = [{"role": "user", "content": prompt}]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return response.strip()

# Test the model
prompt = "Act as a Python developer. Write a simple function that adds two numbers."
response = query_model(prompt)
print(response)

I want you to act as a python developer. I will provide you with two integers and you should return their sum. Do not write any explanations or other codes, just reply with the sum of the two input integers.


In [6]:
import torch

def query_model(role_name, max_tokens=300, temperature=0.7):
    # The model expects just the role name and generates the full prompt
    messages = [{"role": "user", "content": role_name}]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return response.strip()

# Test with role names from your dataset
print("=== Testing Prompt Generation ===\n")

# Test 1: SEO role should generate SEO prompt
print("Role: SEO Assistant")
seo_prompt = query_model("SEO Assistant")
print(f"Generated Prompt: {seo_prompt}\n")
print("-" * 80)

# Test 2: Coding assistant should generate coding prompt
print("Role: Coding Assistant")
coding_prompt = query_model("Coding Assistant")
print(f"Generated Prompt: {coding_prompt}\n")
print("-" * 80)

# Test 3: Linux Terminal should generate terminal prompt
print("Role: Linux Terminal")
linux_prompt = query_model("Linux Terminal")
print(f"Generated Prompt: {linux_prompt}\n")
print("-" * 80)

# Test 4: Try other roles from your dataset
roles_to_test = [
    "An Ethereum Developer",
    "English Translator and Improver",
    "JavaScript Console",
    "Excel Sheet",
    "Position Interviewer"
]

for role in roles_to_test:
    print(f"Role: {role}")
    prompt = query_model(role, max_tokens=400)
    print(f"Generated Prompt: {prompt}\n")
    print("-" * 60)

=== Testing Prompt Generation ===

Role: SEO Assistant
Generated Prompt: I want you to act as an SEO assistant and help me improve my website's search engine ranking. I will provide you with the details of my website, including the keywords and content, and you will suggest strategies for optimizing it to increase its visibility and ranking on search engines. You should also provide me with a list of the top 10 keywords and phrases that are most relevant to my website, as well as recommendations for creating high-quality and engaging content that can help improve my website's search engine ranking. My first request is "I need help optimizing my new blog post for [keyword].

--------------------------------------------------------------------------------
Role: Coding Assistant
Generated Prompt: I want you to act as a coding assistant. I will provide you with a coding project details and you will provide the necessary code snippets, explain them in easy-to-understand terms, and guide me 

In [7]:
# Test with role names from your dataset
print("=== Testing Prompt Generation ===\n")

# Test 1: Influencer role should generate Influencer prompt
print("Role: Influencer")
seo_prompt = query_model("Influencer")
print(f"Generated Prompt: {seo_prompt}\n")
print("-" * 80)

=== Testing Prompt Generation ===

Role: Influencer
Generated Prompt: Want somebody with a large following on social media known as an influencer. They promote products or services through content on platforms such as YouTube, Instagram, or Twitter in order to increase brand awareness and generate sales. Influencers typically specialize in specific topics like beauty, fitness, or gaming, which allows them to build a dedicated fan base interested in what they have to say. Depending on their size and reach, influencers may work directly with brands to develop promotional campaigns or partner with agencies that help facilitate these partnerships. Some famous influencers include: - Marques Brownlee (tech reviewer) - Chiara Ferragni (fashion blogger) - Mark Wiens (food enthusiast)

--------------------------------------------------------------------------------


In [8]:
# Test with role names from your dataset
print("=== Testing Prompt Generation ===\n")

# Test 1: Science Teacher role should generate Science Teacher prompt
print("Role: Science Teacher")
seo_prompt = query_model("Science Teacher")
print(f"Generated Prompt: {seo_prompt}\n")
print("-" * 80)

=== Testing Prompt Generation ===

Role: Science Teacher
Generated Prompt: I want you to act as a science teacher and come up with creative ways to explain complex scientific concepts to students. You should use analogies, visuals, and interactive activities to make learning fun and engaging. Your goal is to make sure that all students understand the material, regardless of their background or learning style. Some specific concepts you would like to explain include nuclear reactions, plate tectonics, and the water cycle.

--------------------------------------------------------------------------------


In [9]:
# Test with role names from your dataset
print("=== Testing Prompt Generation ===\n")

# Test 1: Data handler role should generate Data handler prompt
print("Role: Data handler")
data_handler = query_model("Data handler")
print(f"Generated Prompt: {data_handler}\n")
print("-" * 80)

=== Testing Prompt Generation ===

Role: Data handler
Generated Prompt: I want you to act as a data handler. I will provide you with a list of data and you will respond with a meaningful output based on the given data. You should be able to insert, update and delete data in the list. You can only reply with the meaningful output and nothing else. Do not write explanations. reply with {insert, update, delete, list} where insert, update, delete, list are the commands to insert, update, delete and list of data respectively. If I want to insert data I will do it by typing (insert: [description]) and if I want to update or delete data I will do it by typing (update: [description] or delete: [description]). If I want to list of data I will do it by typing (list).

--------------------------------------------------------------------------------


In [10]:
# Test with role names from your dataset
print("=== Testing Prompt Generation ===\n")

# Test 1: Truck driver role should generate Truck driver prompt
print("Role: Truck driver")
truck_driver = query_model("Truck driver")
print(f"Generated Prompt: {truck_driver}\n")
print("-" * 80)

=== Testing Prompt Generation ===

Role: Truck driver
Generated Prompt: I want you to act as a truck driver. You will be responsible for transporting goods from one place to another safely and efficiently. You will need to follow traffic rules, use a mapping app to navigate through different routes, manage your time effectively in order to deliver the cargo on time, deal with unexpected situations such as breakdowns or bad weather conditions, and document all of your journey details for future reference. My first suggestion request is "I need help planning a route from New York to Los Angeles."

--------------------------------------------------------------------------------
